<a href="https://colab.research.google.com/github/SebastienBienfait/L2C-Data-managment/blob/main/Data_Exploration_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring data sets
---

This [link](https://docs.google.com/document/d/1cijDOCDixsYu-Rr9pC8STPPXado3xoFpgBAZgdDTLHs/edit?usp=sharing) will take you to a list of some of the datasets we use during the Programming for Data course.

For this exercise, please choose a dataset to work with.

Use the code cell below to play with the data.  Look at what data is included, where there is any null data, if there are any columns that need renaming, removing, new values calculated from them.

Form some ideas of what you might want to know from the data and use the techniques you have learned so far to show the answers to these.  You might do this by getting summary statistics, by sorting and showing parts of the dataset, by calculating new values, etc.

### What to do
---

Decide what you want to know, explain this in the TEXT cell below, then write some code to get the answer.  When you have done this, add a new text cell below the code cell, explain what you have found.  Then do the same again, adding text cell, code cell, text cell, for something else you would like to know.

In [37]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd
import numpy as np


#URL = "https://realpython.github.io/fake-jobs/"


url = "https://www.kmfm.co.uk/played/?date=2022-04-28&hour=14"


#df_songs_played = pd.DataFrame(columns=["song_name","artist_name","time_played"])

def convert_datetime(time_str,input,output):
  if time_str == None:
    return float("nan")

  date_form = datetime.strptime(time_str,input)
  time_played = datetime.strftime(date_form, output)
  return time_played

def get_html(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  #print(soup.prettify())
  return soup

def get_genre_and_age(url):
  try:
    if url == None:
      return float("nan"), float("nan")
    elif url != None:
      page = requests.get(url)
      soup = BeautifulSoup(page.content, 'html.parser')

      song_info = soup.find(class_="product-meta typography-callout-emphasized").text.strip()
      song_genre = song_info.split("·")[0]
      #song_age_str = soup.find("p", class_="song-released-container").text.strip()
      song_age_str = song_info.split("·")[1].strip()
      song_age = convert_datetime(song_age_str,"%Y","%Y")
      return song_age, song_genre
  except: # (ValueError) as err:
    return float("nan"), float("nan")



def find_songs(url):
  results = url.find(class_="s-page")


  tracks_played = results.find_all(class_="gm-sec-text")
  tracks_href = results.find_all("a", class_="icon icon-download")

  track_list = []
  for track in tracks_played:

    itunes_url = track.find("a",class_ = "icon icon-download")
    if itunes_url != None:
      Itunes_link = itunes_url["href"]
    else:
      Itunes_link = None
   
    song_age, song_genre = get_genre_and_age( Itunes_link)
    #print(song_age, song_genre)
    try:
      dj_name = results.find("h2").text.strip().split("with ",1)[1].replace(")","") #This finds the DJ name(s) from the "What we played" string and strips all but the DJ name from it. ####"Temp"#####
    except:
      dj_name = float("nan")
    song_name = track.find("p", class_="gm-sec-title").text.strip() #This finds the song name from the first <a class="gm-sec-title">.
    artist_name = track.find("a").text.strip() #This stips the artist name from the url link <a href="/artist/"> which is under the second <a class="gm-sec-title">.
    time_str = track.find("p", class_="gm-sec-meta").text.strip() #This strips the time played from the <a class="gm-sec-meta"> at the top, then converts it into 24hr datetime format below.
    time_played = convert_datetime(time_str,"%I:%M%p","%H:%M")


    #print(song_name)
    #print(artist_name)
    #print(time_played)
    #print("\n")

      
    track_info = [song_name, artist_name, song_genre, song_age, time_played, dj_name]
    track_list.append(track_info)
  #print(itunes_list[1])
  return track_list


#html_text = get_html("https://www.kmfm.co.uk/played/?date=2022-04-28&hour=14")
#data_df = pd.DataFrame(find_songs(html_text),columns=["song_name", "artist_name", "song_genre", "publish_year", "time_played_at", "dj_name"])


def iterate_over_day(daycode):
  df_songs = pd.DataFrame(columns=["song_name", "artist_name", "song_genre", "publish_year", "time_played_at", "dj_name"])
  base_url = "https://www.kmfm.co.uk/played/"
  day_querey = "?date="+ daycode
  actual_querey = "https://www.kmfm.co.uk/played/?date=2022-04-27&hour=04"
  print

  hours_list = ["04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"] #  
  for hour in hours_list:
    hour_querey = "&hour=" + hour
    final_querey = base_url + day_querey + hour_querey
    print(final_querey)
    site_html = get_html(final_querey)
    temp_df = pd.DataFrame(find_songs(site_html), columns=["song_name", "artist_name", "song_genre", "publish_year", "time_played_at", "dj_name"])
    df_songs = pd.concat([df_songs,temp_df], ignore_index=True)
  return df_songs
df_songs_played = iterate_over_day("2022-04-27")


https://www.kmfm.co.uk/played/?date=2022-04-27&hour=06
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=07
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=08
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=09
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=10
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=11
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=12
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=13
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=14
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=15
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=16
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=17
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=18
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=19
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=20
https://www.kmfm.co.uk/played/?date=2022-04-27&hour=21


,song_name,artist_name,song_genre,publish_year,time_played_at,dj_name
0,Stay,The Kid Laroi & Justin Bieber,Pop,2021,06:07,Garry and Claire
1,Green Green Grass,George Ezra,NaN,NaN,06:11,Garry and Claire
2,I Will Never Let You Down,Rita Ora,Pop,2014,06:14,Garry and Claire
3,Giant,Calvin Harris / Rag N Bone Man,Dance,2019,06:23,Garry and Claire
4,High Hopes,Panic! At The Disco,Alternative,2018,06:35,Garry and Claire
...,...,...,...,...,...,...
179,Run,Becky Hill & Galantis,Dance,2022,21:35,Numi
180,Heat Waves,Glass Animals,Pop,2020,21:44,Numi
181,Flowers,Lauren Spencer Smith,Pop,2022,21:48,Numi
182,What Would You Do?,"Joel Corry, David Guetta & Bryson Tiller",Dance,2022,21:50,Numi


In [39]:
URL = "https://www.kmfm.co.uk/played/?date=2022-04-27&hour=04"



#df_songs_played = pd.DataFrame(data,columns=["song_name", "artist_name", "song_genre", "publish_year", "time_played_at", "dj_name"])
#df_songs_played = pd.concat([df_songs_played,])
display(df_songs_played)



,song_name,artist_name,song_genre,publish_year,time_played_at,dj_name
0,Stay,The Kid Laroi & Justin Bieber,Pop,2021,06:07,Garry and Claire
1,Green Green Grass,George Ezra,NaN,NaN,06:11,Garry and Claire
2,I Will Never Let You Down,Rita Ora,Pop,2014,06:14,Garry and Claire
3,Giant,Calvin Harris / Rag N Bone Man,Dance,2019,06:23,Garry and Claire
4,High Hopes,Panic! At The Disco,Alternative,2018,06:35,Garry and Claire
...,...,...,...,...,...,...
179,Run,Becky Hill & Galantis,Dance,2022,21:35,Numi
180,Heat Waves,Glass Animals,Pop,2020,21:44,Numi
181,Flowers,Lauren Spencer Smith,Pop,2022,21:48,Numi
182,What Would You Do?,"Joel Corry, David Guetta & Bryson Tiller",Dance,2022,21:50,Numi


In [129]:
#date_form = datetime.strptime(time_played,"%I:%M%p")
#outtime = datetime.strftime(date_form, "%H:%M")
datetime_object = datetime.strptime('24-3-2018 1:33PM', '%I:%M%p')
print(datetime_object)
list_time = list(time_played)
print(type(list_time[0]))
print(list_time[::])

ValueError: ignored

In [172]:

#print(soup.prettify())



#teststring = "What we played on Thursday, April 28th at 1:00pm (Daytimes with Andy Walker)"
#stripped = teststring.split("with ",1)[1].replace(")","")
#replaced = stripped.replace(")","")
#print((stripped))
test_var = None
print(type(test_var) == "NoneType")

False
